# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20901997


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:03<00:46,  1.54s/it]

Rendering models:   9%|▉         | 3/32 [00:03<00:34,  1.19s/it]

Rendering models:  16%|█▌        | 5/32 [00:03<00:24,  1.12it/s]

Rendering models:  19%|█▉        | 6/32 [00:04<00:17,  1.48it/s]

...

Rendering models:  91%|█████████ | 29/32 [00:16<00:01,  2.75it/s]

Rendering models:  94%|█████████▍| 30/32 [00:16<00:00,  2.88it/s]

Rendering models:  97%|█████████▋| 31/32 [00:16<00:00,  2.77it/s]

Rendering models: 100%|██████████| 32/32 [00:17<00:00,  2.70it/s]

klmasters                             0.000174
CTidwell3                             0.000518
Devlin.D                              0.000266
not-logged-in-bc8d75575cbd7912514f    0.004109
dkehne19491                           0.038837
Kristhope                             0.000185
not-logged-in-60953d21f1d9f596b2fe    0.000203
not-logged-in-a9d10e6d6d9f82971566    0.000235
not-logged-in-9c310346c8df6cdafc65    0.004723
not-logged-in-eacff73804c974d0c437    0.000294
not-logged-in-b48f5b396bb5986801e0    0.001133
not-logged-in-ea8ed61a7274cfb6a43b    1.955838
raisamaduro                           0.010989
ElisabethB                            0.000188
Lavadude                              0.016047
not-logged-in-48c45b800545e7c29728    0.000914
sirtylerli                            0.000218
jmacd297                              0.001483
h4ranek                               0.001193
not-logged-in-b44456267c682965db69    0.007849
spannerfarm                           0.000180
spannerfarm  

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())